In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [2]:
combined_data = pd.read_csv('english.csv')
combined_data['features'] = combined_data['overview'].fillna('').astype(str) + ' ' + combined_data['genres'].fillna('').astype(str)+ ' ' + combined_data['vote_average'].fillna('').astype(str) +  ' ' + combined_data['popularity'].fillna('').astype(str)

In [3]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(combined_data['features'])
tfidf_models = (tfidf, tfidf_matrix)

In [4]:
nn_model = NearestNeighbors(n_neighbors=6, algorithm='auto')
nn_model.fit(tfidf_models[1])
nn_models = nn_model

In [5]:
def get_recommendations_with_overviews(movie_name, tfidf_models=tfidf_models, nn_models=nn_models, combined_data=combined_data):
    tfidf, tfidf_matrix = tfidf_models
    nn_model = nn_models
    movie_features = combined_data[combined_data['title'] == movie_name]['features'].iloc[0]
    movie_tfidf = tfidf.transform([movie_features])
    distances, indices = nn_model.kneighbors(movie_tfidf)
    movie_indices = indices.flatten()[1:]
    recommendations_with_overviews = []
    for idx in movie_indices:
        movie_name = combined_data['title'].iloc[idx]
        movie_overview = combined_data['overview'].iloc[idx]
        recommendations_with_overviews.append(movie_name)
    return recommendations_with_overviews

In [ ]:
from flask import Flask, render_template, request

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/recommendations', methods=['POST'])
def recommendations():
    movie_name = request.form['movie_name']
    recommendations = get_recommendations_with_overviews(movie_name)
    return render_template('recommendations.html', recommendations=recommendations)

if __name__ == '__main__':
    app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [13/Apr/2024 13:22:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2024 13:22:23] "POST /recommendations HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2024 13:24:35] "POST /recommendations HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2024 13:26:04] "POST /recommendations HTTP/1.1" 200 -


In [36]:
movie_name = 'Wedding Crashers'
recommendations_with_overviews = get_recommendations_with_overviews(movie_name)
for movie in recommendations_with_overviews:
    print(movie['name'])
    if pd.notnull(movie['overview']):
        print(movie['overview'])
    print()

Bride Wars
Two best friends become rivals when their respective weddings are accidentally booked for the same day.

Forsaken
John Henry returns to his hometown in hopes of repairing his relationship with his estranged father, but a local gang is terrorizing the town. John Henry is the only one who can stop them, however he has abandoned both his gun and reputation as a fearless quick-draw killer.

I Still Believe
The true-life story of Christian music star Jeremy Camp and his journey of love and loss that looks to prove there is always hope.

What Lies Beneath
When Claire Spencer starts hearing ghostly voices and seeing spooky images, she wonders if an otherworldly spirit is trying to contact her. All the while, her husband tries to reassure her by telling her it's all in her head. But as Claire investigates, she discovers that the man she loves might know more than he's letting on.

Dance with Me
Young Cuban Rafael just buried his mother, and comes to Houston to meet his father John f